# 使用预训练模型进行图像分类
[这篇教程](https://learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/)是PyTorch实战篇的第一篇。在这里我们将使用Pytorch内置的预训练模型解决**图像分类**问题。
预训练模型是在大的benchmark数据集（如ImageNet)训练好的神经网络模型。
在导入torchvision.models后，通过dir可以看到PyTorch内置了15个类的网络模型。其中首字母大写的为类，全部小写的为返回这些模型实例的函数。

In [5]:
from torchvision import models
import torch

## 使用AlexNet进行图像分类
![avatar](.asset/AlexNet-1.png)

### 第一步：加载模型
通过models中的函数alexnet加载模型，通过指定pretrained参数为真来使用预训练过模型。

In [6]:
from torchinfo import summary
batch_size = 64
alexnet = models.alexnet(pretrained=True)
summary(alexnet, input_size=[batch_size,3,227,227])

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /Users/liushuduo/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100.0%


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  --                        --
├─Sequential: 1-1                        [64, 256, 6, 6]           --
│    └─Conv2d: 2-1                       [64, 64, 56, 56]          23,296
│    └─ReLU: 2-2                         [64, 64, 56, 56]          --
│    └─MaxPool2d: 2-3                    [64, 64, 27, 27]          --
│    └─Conv2d: 2-4                       [64, 192, 27, 27]         307,392
│    └─ReLU: 2-5                         [64, 192, 27, 27]         --
│    └─MaxPool2d: 2-6                    [64, 192, 13, 13]         --
│    └─Conv2d: 2-7                       [64, 384, 13, 13]         663,936
│    └─ReLU: 2-8                         [64, 384, 13, 13]         --
│    └─Conv2d: 2-9                       [64, 256, 13, 13]         884,992
│    └─ReLU: 2-10                        [64, 256, 13, 13]         --
│    └─Conv2d: 2-11                      [64, 256, 13, 13]        

## 第二步：指定图像变换
为了使模型发挥最好的作用，我们需要先对模型进行预处理。这个过程一般包括图形的resize（把大小调整为网络的输入格式）、Crop（裁剪图像）、ToTensor（转换为pytorch的数据结构）、Normalization（归一化）等等。
在这里，我们归一化使用的均值和方差是ImageNet所有图像的均值和方差。通过归一化操作可以把图片的值分布转换为标准正态分布。

In [7]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize( 
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

## 第三步：接下来我们将使用AlexNet识别一张狗的照片
![avatar](./dog.jpg)

In [8]:
# Load Image
from PIL import Image
img = Image.open('dog.jpg')
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

# Model inference
alexnet.eval()
out = alexnet(batch_t)
print(out.shape)

torch.Size([1, 1000])


可以看到模型输出了`[1,1000]`的tensor，接下来要把tensor转换为我们希望得到的目标种类。

In [10]:
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

_, index = torch.max(out, 1)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
print(classes[index[0]], percentage[index[0]].item())

258, Samoyed 66.32768249511719
